# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys2 import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Set path to csv file
city_weather_path = "../output_data/CityWeather.csv"

#Read in CityWeather csv file and display first 10 rows to verify it read correctly
city_weather_df = pd.read_csv(city_weather_path)
city_weather_df.head(10)


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,hermanus,-34.4187,19.2345,67.80,80,12,17.20
1,hobart,-42.8794,147.3294,49.50,74,100,9.22
2,san miguelito,9.0333,-79.5000,85.21,75,40,11.50
3,cape town,-33.9258,18.4232,65.55,76,15,18.01
4,mataura,-46.1927,168.8643,50.65,96,100,7.45
5,ushuaia,-54.8000,-68.3000,47.86,76,75,25.32
6,iqaluit,63.7506,-68.5145,31.69,84,99,15.01
7,busselton,-33.6500,115.3333,55.26,90,97,8.37
8,la peca,-5.6111,-78.4350,73.24,84,100,3.36
9,port hardy,50.6996,-127.4199,45.07,80,83,9.89


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
#Store latitude and longitude for cities in locations
locations = city_weather_df[["Latitude", "Longitude"]].astype(float)

#Store city humidity levels as humidity
humidity = city_weather_df["Humidity"]


In [5]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
#Create lists to keep only data from potential vacation spots
vacation_city = []
vacation_lat = []
vacation_long = []

#Loop through original dataframe and keep only the cities that match good vacation weather
#I'm defining good vacation weather as temperature between 76 - 85 degrees, humidity between 30 and 60 percent,
#and cloudiness less than 20 percent
for index, row in city_weather_df.iterrows():
    if(row["Temperature"] > 76) and (row["Temperature"] < 85) and (row["Humidity"] > 30) and (row["Humidity"] < 60) and (row["Cloudiness"] < 20):
        vacation_city.append(row["City"])
        vacation_lat.append(row["Latitude"])
        vacation_long.append(row["Longitude"])

vacation_cities_df = pd.DataFrame({"City": vacation_city, "Latitude": vacation_lat, "Longitude": vacation_long})        
vacation_cities_df

,City,Latitude,Longitude
0,dahanu,19.9667,72.7333
1,abu kamal,34.4506,40.9171
2,bhuj,23.2667,69.6667
3,cocorit,27.5833,-109.9667
4,kirkuk,35.4681,44.3922
5,sur,22.5667,59.5289
6,kontagora,10.3999,5.4695
7,lincoln,40.8000,-96.6670
8,fruitville,27.3298,-82.4576
9,palghar,19.6833,72.7500


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
#make one call to google api to see the response
#Set variables for parameters
target_search = "hotel"
target_radius = 5000
target_location = "40.8000,-96.6670"
#Set parameters
params = {
    "location": target_location,
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_atlanta = requests.get(base_url, params=params)
hotel_atlanta = hotel_atlanta.json()
print(json.dumps(hotel_atlanta, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 40.8336795,
                    "lng": -96.692145
                },
                "viewport": {
                    "northeast": {
                        "lat": 40.83499592989272,
                        "lng": -96.69076302010728
                    },
                    "southwest": {
                        "lat": 40.83229627010728,
                        "lng": -96.69346267989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "The Scarlet, Lincoln, a Tribute Portfolio Hotel",
            "photos": [
                {
    

In [23]:
hotel_lat = hotel_lat_long["results"][0]["geometry"]["location"]["lat"]
hotel_lat

19.6923849

In [24]:
#Create lists to store the info returned from google api
hotel_name = []
hotel_lat = []
hotel_long = []

#Set variables for parameters
target_search = "hotel"
target_radius = 5000

#Set parameters
params = {
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
}

#Loop through the vacation_cities_df and run a search for hotels within a 5000 meter radius of each city
for index, row in vacation_cities_df.iterrows():
    #base_url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row['Latitude']
    long = row['Longitude']
    
    #Update params to include city location
    params['location'] = f"{lat},{long}"
    
    #Make request from google places api
    hotel_lat_long = requests.get(base_url, params=params)
    
    #convert to json
    hotel_lat_long = hotel_lat_long.json()
    
    #Save the hotel name and location to put into a dataframe
    hotel_name.append(hotel_lat_long["results"][0]["name"])
    hotel_lat.append(hotel_lat_long["results"][0]["geometry"]["location"]["lat"])
    hotel_long.append(hotel_lat_long["results"][0]["geometry"]["location"]["lng"])
    
#Create a hotel_df dataframe with the results of the search
hotel_df = pd.DataFrame({"Hotel Name": hotel_name, "Lat": hotel_lat, "Lng": hotel_long})
hotel_df

,Hotel Name,Lat,Lng
0,Seawoods Beach Hotel,19.992271,72.723227
1,البوكمال,34.451844,40.911932
2,Regenta Resort Bhuj,23.234930,69.645711
3,Ferrari Motel,27.544650,-109.927829
4,Kirkuk Plaza Hotel,35.458940,44.382163
5,Sur Plaza Hotel,22.566435,59.495858
6,Hard Resource lodge,10.406102,5.483988
7,"The Scarlet, Lincoln, a Tribute Portfolio Hotel",40.833680,-96.692145
8,Hampton Inn Sarasota - I-75 Bee Ridge,27.300040,-82.450028
9,Hotel Sai Residency,19.692385,72.760874


In [27]:
hotel_df["City"] = vacation_cities_df["City"]
hotel_df["Country"] = ["India", "Syria", "India", "Mexico", "Iraq", "Oman", "Nigeria", "United States", "United States", "India"]
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Seawoods Beach Hotel,19.992271,72.723227,dahanu,India
1,البوكمال,34.451844,40.911932,abu kamal,Syria
2,Regenta Resort Bhuj,23.234930,69.645711,bhuj,India
3,Ferrari Motel,27.544650,-109.927829,cocorit,Mexico
4,Kirkuk Plaza Hotel,35.458940,44.382163,kirkuk,Iraq
5,Sur Plaza Hotel,22.566435,59.495858,sur,Oman
6,Hard Resource lodge,10.406102,5.483988,kontagora,Nigeria
7,"The Scarlet, Lincoln, a Tribute Portfolio Hotel",40.833680,-96.692145,lincoln,United States
8,Hampton Inn Sarasota - I-75 Bee Ridge,27.300040,-82.450028,fruitville,United States
9,Hotel Sai Residency,19.692385,72.760874,palghar,India


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [34]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(locations, fill_color='black', stroke_color='black', scale=3, info_box_content=hotel_info)

fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))